In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [ ]:
!pip install fastcoref
# or for training:


In [69]:
text = """The European Central Bank (ECB) is the central component of the Eurosystem and the European System of Central Banks (ESCB) as well as one of seven institutions of the European Union.[2] It is one of the world's most important central banks with a balance sheet total of around 7 trillion.[3]

The ECB Governing Council makes monetary policy for the Eurozone and the European Union, administers the foreign exchange reserves of EU member states, engages in foreign exchange operations, and defines the intermediate monetary objectives and key interest rate of the EU. The ECB Executive Board enforces the policies and decisions of the Governing Council, and may direct the national central banks when doing so.[4] The ECB has the exclusive right to authorise the issuance of euro banknotes. Member states can issue euro coins, but the volume must be approved by the ECB beforehand. The bank also operates the T2 (RTGS) payments system.

The ECB was established by the Treaty of Amsterdam in May 1999 with the purpose of guaranteeing and maintaining price stability. On 1 December 2009, the Treaty of Lisbon became effective and the bank gained the official status of an EU institution. When the ECB was created, it covered a Eurozone of eleven members. Since then, Greece joined in January 2001, Slovenia in January 2007, Cyprus and Malta in January 2008, Slovakia in January 2009, Estonia in January 2011, Latvia in January 2014, Lithuania in January 2015 and Croatia in January 2023.[5] The current president of the ECB is Christine Lagarde. Seated in Frankfurt, Germany, the bank formerly occupied the Eurotower prior to the construction of its new seat.

The ECB is directly governed by European Union law. Its capital stock, worth €11 billion, is owned by all 27 central banks of the EU member states as shareholders.[6] The initial capital allocation key was determined in 1998 on the basis of the states' population and GDP, but the capital key has been readjusted since.[6] Shares in the ECB are not transferable and cannot be used as collateral"""

In [82]:
from fastcoref import spacy_component
import spacy


nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("fastcoref")


/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=dev

In [21]:
import spacy

def analyze_text_complexity(text):
    doc = nlp(text)

    total_words = len([token.text for token in doc if token.is_alpha])
    total_sentences = len(list(doc.sents))
    total_chars = sum(len(token.text) for token in doc)
    unique_words = len(set(token.text.lower() for token in doc if token.is_alpha))

    content_words_noun = [token for token in doc if token.pos_ in {"NOUN",}]
    content_words_verb = [token for token in doc if token.pos_ in {"VERB"}]
    content_words_adv = [token for token in doc if token.pos_ in {"ADV"}]
    content_words_adj = [token for token in doc if token.pos_ in {"ADJ"}]

    lexical_density_noun = round(len(content_words_noun) / total_words,4) if total_words > 0 else 0
    lexical_density_verb = round(len(content_words_verb) / total_words,4) if total_words > 0 else 0
    lexical_density_adv = round(len(content_words_adv) / total_words,4) if total_words > 0 else 0
    lexical_density_adj = round(len(content_words_adj) / total_words,4) if total_words > 0 else 0

    ttr = unique_words / total_words if total_words > 0 else 0
    avg_word_length = total_chars / total_words if total_words > 0 else 0
    avg_sentence_length = total_words / total_sentences if total_sentences > 0 else 0

    # Syntactic Depth
    max_depth = max([token.i - token.head.i for token in doc if token.head != token]) if doc else 0

    # Count clauses (number of conjunctions as an approximation)
    clause_count = sum(1 for token in doc if token.dep_ in {"conj", "ccomp", "advcl"})

    # Named Entity Count
    named_entities = len(doc.ents)

    return {
        "Lexical Density Noun": lexical_density_noun,
        "Lexical Density Verb": lexical_density_verb,
        "Lexical Density Adverb": lexical_density_adv,
        "Lexical Density Adjective": lexical_density_adj,
        "Type-Token Ratio": ttr,
        "Average Word Length": avg_word_length,
        "Average Sentence Length": avg_sentence_length,
        "Max Syntactic Depth": max_depth,
        "Clauses per Sentence": clause_count / total_sentences if total_sentences > 0 else 0,
        "Named Entity Count": named_entities    }

# Example usage
metrics = analyze_text_complexity(text)
print(metrics)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

{'Lexical Density Noun': 0.1881, 'Lexical Density Verb': 0.0846, 'Lexical Density Adverb': 0.0345, 'Lexical Density Adjective': 0.0627, 'Type-Token Ratio': 0.4952978056426332, 'Average Word Length': 5.382445141065831, 'Average Sentence Length': 16.789473684210527, 'Max Syntactic Depth': 40, 'Clauses per Sentence': 1.368421052631579, 'Named Entity Count': 56}


In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer


# Text to extract triplets from



In [86]:
def extract_facts(text):
  prs = text.split('\n\n')
  new_p = []
  for p in prs:
    doc = nlp(p,component_cfg={"fastcoref": {'resolve_text': True}})
    new_p.append(doc._.resolved_text)
  all_snts=[]
  for p in new_p:
    doc = nlp(p)
    for s in doc.sents:
      all_snts.append(s.text)
  facts_all=[]
  i=0
  tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
  model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
  for t in all_snts:
    i=i+1
    model_inputs = tokenizer(t, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

    # Generate
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
    )

    # Extract text
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    facts_sent = []
    # Extract triplets
    for idx, sentence in enumerate(decoded_preds):
        trip = extract_triplets(sentence)
        for tr in trip:
          facts_sent.append({'head':tr['head'],'type':tr['type'],'tail':tr['tail']})
    facts_all.append({f'sentence {i}':facts_sent})

  return facts_all


In [87]:
extract_facts(text)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[{'sentence 1': [{'head': 'European Central Bank',
    'type': 'part of',
    'tail': 'Eurosystem'},
   {'head': 'European Central Bank',
    'type': 'part of',
    'tail': 'European System of Central Banks'},
   {'head': 'Eurosystem', 'type': 'has part', 'tail': 'European Central Bank'},
   {'head': 'European System of Central Banks',
    'type': 'has part',
    'tail': 'European Central Bank'}]},
 {'sentence 2': [{'head': 'European Central Bank',
    'type': 'instance of',
    'tail': 'central bank'}]},
 {'sentence 3': [{'head': 'Eurozone',
    'type': 'shares border with',
    'tail': 'European Union'},
   {'head': 'European Union',
    'type': 'shares border with',
    'tail': 'Eurozone'}]},
 {'sentence 4': [{'head': 'ECB Executive Board',
    'type': 'part of',
    'tail': 'The ECB Governing Council'},
   {'head': 'The ECB Governing Council',
    'type': 'has part',
    'tail': 'ECB Executive Board'}]},
 {'sentence 5': [{'head': 'euro banknotes',
    'type': 'owned by',
    'tail'

In [71]:
import spacy

# Load SpaCy model

def preprocess_text(text):
    """
    Preprocess the text using spaCy: tokenize, remove stopwords, and lemmatize.
    :param text: Input sentence
    :return: Set of lemmatized tokens (without stopwords)
    """
    # Process the text with spaCy
    doc = nlp(text.lower())  # Lowercase and tokenize
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return set(tokens)

def modified_jaccard_similarity(question, answer):
    """
    Calculate modified Jaccard similarity between a question and an answer using spaCy preprocessing.
    :param question: Input question text
    :param answer: Input answer text
    :return: Modified Jaccard similarity score
    """
    # Preprocess the question and answer to extract meaningful tokens
    question_tokens = preprocess_text(question)
    answer_tokens = preprocess_text(answer)

    # Compute the intersection and union of the token sets
    intersection = question_tokens.intersection(answer_tokens)
    union = question_tokens.union(answer_tokens)

    # Return the Jaccard similarity (ratio of intersection and union sizes)
    return len(intersection) / len(union) if len(union) > 0 else 0

# Example usage
question = "What is the capital of France?"
answer = "Paris is the capital city of France."

jaccard_score = modified_jaccard_similarity(question, answer)
print(f"Modified Jaccard Similarity: {jaccard_score:.3f}")


Modified Jaccard Similarity: 0.500


In [ ]:
!pip install tiktoken

In [68]:
import tiktoken
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sentence_transformers import CrossEncoder
import spacy
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# Initialize tiktoken encoder for GPT-3 (or GPT-4) models
enc = tiktoken.get_encoding("cl100k_base")  # Use appropriate model encoding

# Initialize spaCy for NER
nlp = spacy.load("en_core_web_trf")

# Initialize a CrossEncoder model for direct sentence pair matching
def lemma_overlap(context,answer):
    return modified_jaccard_similarity(context, answer)

def tokenize_text(text):
    """Tokenize the input text using tiktoken."""
    return enc.encode(text)

def extract_entities(text):
    """Extract named entities using spaCy's NER."""
    doc = nlp(text)
    return set(ent.text for ent in doc.ents)

def compute_token_overlap(question, answer, context):
    """Calculate token overlap between the context, question, and answer."""
    context_tokens = set(tokenize_text(context))
    question_tokens = set(tokenize_text(question))
    answer_tokens = set(tokenize_text(answer))

    # Token overlap between context and answer
    context_answer_overlap = len(context_tokens.intersection(answer_tokens)) / len(context_tokens) if len(context_tokens) > 0 else 0
    # Token overlap between question and answer
    question_answer_overlap = len(question_tokens.intersection(answer_tokens)) / len(question_tokens) if len(question_tokens) > 0 else 0

    # Return the average overlap score
    return {'context_overlap':context_answer_overlap,'question_overlap':question_answer_overlap}

def compute_nli_score(context, answer):
    """Check if the answer entails the question using an NLI model."""
    model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    premise = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
    hypothesis = "Emmanuel Macron is the President of France"

    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].to('cpu'))  # device = "cuda:0" or "cpu"
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    label_names = ["entailment", "neutral", "contradiction"]
    prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}

    return prediction

def improved_factual_matching(question, answer, context):
    """Combines token overlap, semantic similarity, and NLI for improved matching."""

    # 1. Token Overlap
    token_overlap_score = compute_token_overlap(question, answer, context)

    # 2. Cross-Encoder Similarity (semantic matching)

    # 3. Named Entity Recognition (NER) - Check for entities in the question that must appear in the answer
    question_entities = extract_entities(question)
    answer_entities = extract_entities(answer)
    entity_overlap = len(question_entities.intersection(answer_entities)) / len(question_entities) if len(question_entities) > 0 else 0

    # 4. Natural Language Inference (NLI) - Check if the answer entails the question
    label = compute_nli_score(question, answer)
    root_overlap = lemma_overlap(question,answer)
    # Combine all scores into a final weighted score

    return {
        'token_overlap_score': token_overlap_score,
        'entity_overlap': entity_overlap,
        'context_nli_label': label,
        'lemma_overlap':root_overlap
    }

# Example usage
context = "Paris is the capital of France. It is known for its art, fashion, and culture."
question = "What is the capital of France?"
answer = "The capital of France is Paris."

matching_results = improved_factual_matching(question, answer, context)

# Display the results
print(f"Token Overlap Score: {matching_results['token_overlap_score']}")
print(f"Entity Overlap: {matching_results['entity_overlap']:.3f}")
print(f"NLI Label (Entailment/Contradiction): {matching_results['context_nli_label']}")
print(f"Lemma overlap: {matching_results['lemma_overlap']}")


Token Overlap Score: {'context_overlap': 0.3125, 'question_overlap': 0.5714285714285714}
Entity Overlap: 1.000
NLI Label (Entailment/Contradiction): {'entailment': 82.4, 'neutral': 17.3, 'contradiction': 0.3}
Lemma overlap: 0.6666666666666666
